In [1]:
import numpy as np
emb=np.load("emb.npy")
vi=np.load("text_prob.npy")
wi=np.load("cluter_prob.npy")

In [3]:
f=open("d:/deep/sis/annotation.txt")
text=f.readlines()
words=[]
dic=set()
for line in text:
    word=line.split()
    for w in word:
        if w.isalnum():
            dic.add(w)
            words.append(w)
f.close()

In [4]:
f=open("d:/deep/caption/annotation.txt")
text=f.readlines()
for line in text:
    word=line.split()
    for w in word:
        if w.isalnum():
            dic.add(w)
            words.append(w)
f.close()

In [5]:
f=open("d:/deep/annotations/captions.txt")
text=f.readlines()
for line in text:
    word=line.split()
    for w in word:
        w="".join(l for l in w if l not in {',','.','!','?','\"','(',')','\''})
        if len(w)>0:
            dic.add(w)
            words.append(w)
f.close()

In [6]:
import collections
import math
def build_dataset(words, vocabulary_size):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [8]:
data, count, dictionary, reverse_dictionary = build_dataset(words, len(dic))

In [9]:
import pulp
prob=pulp.LpProblem("Optimization Problem",pulp.LpMinimize)
weights=pulp.LpVariable.dicts("weight",(category,dictionary),lowBound=0.0,upBound=1.0)
for i in category:
    for j in dictionary:
        prob+=abs(weights[i][j]*vi[i][j]-wi[i][j])
for j in dictionary:
    prob+=pulp.lpSum([weights[i][j] for i in category])==1
prob.solve()
res={}
for i in category:
    for j in dictionary:
        res[(i,j)]=weights[i][j].value()


In [11]:
np.save('weights.npy',res)